In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [10]:
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()


using cached model
using cached model


In [11]:
import pandas as pd
df=pd.read_csv('continuous_dataset.CSV',encoding='cp949')
df['emotion']=df['emotion'].replace({'ㄴ중립':'중립','ㅈ중립':'중립','ㅍ':'중립','줄':'중립','분':'분노','분ㄴ':'분노','중림':'중립'})
df.dropna(inplace=True)
df

,id,sentences,emotion
0,1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
1,2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
2,3,손님 왔어요.,중립
3,4,손님? 누구?,중립
4,5,몰라요. 팀장님 친구래요.,중립
...,...,...,...
55623,55624,얘긴 다 끝났냐? 원예부,중립
55624,55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55625,55626,여자 숨겨달라는거면 사절이다.,중립
55626,55627,아무래도 안되나요?,중립


In [12]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
encoder.fit(df['emotion'])
df['emotion']=encoder.transform(df['emotion'])
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))   
mapping

{0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복', 6: '혐오'}

In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state=99)
print("train shape is:", len(train))
print("test shape is:", len(test))


train shape is: 38934
test shape is: 16687


In [14]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


using cached model


In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform(i) for i in dataset["sentences"]]
        self.labels = [np.int32(i) for i in dataset["emotion"]]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [17]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      


In [20]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [21]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [22]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    

C:\Users\KIMJOONHONG\Anaconda3\envs\JH\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/1217 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8786253929138184 train acc 0.40625
epoch 1 batch id 201 loss 0.795041561126709 train acc 0.7465796019900498
epoch 1 batch id 401 loss 1.0879604816436768 train acc 0.7642612219451371
epoch 1 batch id 601 loss 0.5819286108016968 train acc 0.7722025790349417
epoch 1 batch id 801 loss 0.882996678352356 train acc 0.7733692259675405
epoch 1 batch id 1001 loss 0.8579422235488892 train acc 0.7763798701298701
epoch 1 batch id 1201 loss 0.7608823776245117 train acc 0.7774510824313072
epoch 1 train acc 0.7777251251213864


  0%|          | 0/1217 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.8385952115058899 train acc 0.8125
epoch 2 batch id 201 loss 0.7195555567741394 train acc 0.7938432835820896
epoch 2 batch id 401 loss 1.0912083387374878 train acc 0.7879519950124688
epoch 2 batch id 601 loss 0.5350221991539001 train acc 0.7880615640599001
epoch 2 batch id 801 loss 0.8364973664283752 train acc 0.7853074282147315
epoch 2 batch id 1001 loss 0.8949173092842102 train acc 0.7859328171828172
epoch 2 batch id 1201 loss 0.7695795893669128 train acc 0.7854392173189009
epoch 2 train acc 0.7856082393366699


...

epoch 3 batch id 1 loss 0.7407984733581543 train acc 0.8125
epoch 3 batch id 201 loss 0.7594726085662842 train acc 0.7938432835820896
epoch 3 batch id 401 loss 1.0667940378189087 train acc 0.7879519950124688
epoch 3 batch id 601 loss 0.47804343700408936 train acc 0.7880615640599001
epoch 3 batch id 801 loss 0.8474216461181641 train acc 0.7852684144818977
epoch 3 batch id 1001 loss 0.8462604880332947 train acc 0.7859015984015985
epoch 3 batch id 1201 loss 0.749754786491394 train acc 0.7854131973355537
epoch 3 train acc 0.7855825614402032


  0%|          | 0/1217 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.7569084763526917 train acc 0.8125
epoch 4 batch id 201 loss 0.763266384601593 train acc 0.7938432835820896
epoch 4 batch id 401 loss 1.040627121925354 train acc 0.7879519950124688
epoch 4 batch id 601 loss 0.4951261878013611 train acc 0.7880615640599001
epoch 4 batch id 801 loss 0.8417426347732544 train acc 0.7852684144818977
epoch 4 batch id 1001 loss 0.8766649961471558 train acc 0.7859015984015985
epoch 4 batch id 1201 loss 0.7185437083244324 train acc 0.7854131973355537
epoch 4 train acc 0.7855825614402032


  0%|          | 0/1217 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.7891342639923096 train acc 0.8125
epoch 5 batch id 201 loss 0.7407046556472778 train acc 0.7938432835820896
epoch 5 batch id 401 loss 1.0818774700164795 train acc 0.7879519950124688
epoch 5 batch id 601 loss 0.5007203817367554 train acc 0.7880615640599001
epoch 5 batch id 801 loss 0.8549792170524597 train acc 0.7852684144818977
epoch 5 batch id 1001 loss 0.8932658433914185 train acc 0.7859015984015985
epoch 5 batch id 1201 loss 0.7293298840522766 train acc 0.7854131973355537
epoch 5 train acc 0.7855825614402032


In [28]:
data_my = {'sentences':'너 진짜 뒤지고싶냐?', "emotion": [2]}
unseen_test = pd.DataFrame(data_my)

unseen_values = unseen_test.values
test_set = BERTDataset(unseen_test, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=0)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    print(out)
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

C:\Users\KIMJOONHONG\Anaconda3\envs\JH\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-3.0091,  0.8569,  0.5748, -0.0365,  3.1797, -0.7186, -2.2343]],
       device='cuda:0', grad_fn=<AddmmBackward>)


C:\Users\KIMJOONHONG\Anaconda3\envs\JH\lib\site-packages\ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/522 [00:00<?, ?it/s]

epoch 5 test acc 3.9600095785440614
